In [4]:
!pip install ipynb

Defaulting to user installation because normal site-packages is not writeable


In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from math import sqrt
from numpy import split
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras_tuner import RandomSearch
from keras_tuner import Hyperband
from keras_tuner import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression

In [35]:
%run Funciones_Temp.ipynb

In [8]:
datos = pd.read_csv("data-room-10T-all.csv", sep = ";", index_col=0)

In [36]:
edificio = ['A','B','C']
n_obs_antes = 6
n_obs_despues = 12
datos_all_rooms = find_events(datos, edificio, n_obs_antes, n_obs_despues)

In [37]:
datos_all_rooms.describe()

,V2,V4,tmed,hrmed,radmed,vvmed,dvmed,prec,dewpt,Hora_1,Hora_2,Hora_3,Season_1,Season_2,Season_3,Season_4
count,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000,9702.000000
mean,28.187528,0.333333,25.235457,42.559380,444.266646,1.449320,151.341053,0.004638,10.290785,0.524222,0.467223,0.008555,0.100186,0.155844,0.513915,0.230056
std,2.515461,0.471429,7.805285,16.577791,306.691012,0.744501,99.483060,0.086860,5.933893,0.499439,0.498950,0.092101,0.300263,0.362726,0.499832,0.420890
min,15.500000,0.000000,5.200000,4.200000,0.000000,0.000000,0.000000,0.000000,-9.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.500000,0.000000,18.500000,31.500000,170.200000,0.900000,63.500000,0.000000,5.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,28.500000,0.000000,27.000000,40.100000,441.150000,1.400000,131.500000,0.000000,11.600000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,30.100000,1.000000,30.900000,52.200000,711.000000,2.000000,217.300000,0.000000,15.100000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,35.700000,1.000000,44.000000,89.800000,1172.000000,4.400000,359.900000,4.600000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
features = feature_selection(10, datos_all_rooms)
datos_all_rooms = datos_all_rooms.loc[:,features]
print("Selected:", features)

Selected: ['V2', 'tmed', 'hrmed', 'vvmed', 'dvmed', 'dewpt', 'Hora_1', 'Hora_2', 'Season_1', 'Season_3', 'Season_4']


In [ ]:
edificio = ['A','B','C']
n_obs_antes = 3
n_obs_despues = 6
datos_all_rooms = find_events(datos, edificio, n_obs_antes, n_obs_despues)

interval = n_obs_antes + n_obs_despues

df = datos_all_rooms
df = shuffle_df(df, interval)

# split en train y test
per_train = 0.75
train, test = split_dataset(df.values, per_train, interval)
print('shape: [muestras, pasos de tiempo, características]')
print('train shape:', train.shape)
print('test shape:', test.shape)

n_input = n_obs_antes
n_out = 1

train_x, train_y = to_supervised(train, n_input, n_out)
test_x, test_y = to_supervised(test, n_input, n_out)
print('train_x shape:' , train_x.shape) 
print('train_y shape:'  , train_y.shape)     
print('test_x shape:' , test_x.shape)  
print('test_y shape:' , test_y.shape)   

rows_to_keep = n_obs_despues 
rows_to_drop = n_obs_antes
period = rows_to_keep + rows_to_drop

train_sel = (np.arange(len(train_x)) % period) < rows_to_keep
test_sel = (np.arange(len(test_x)) % period) < rows_to_keep

train_x = train_x[train_sel,:,:]
train_y = train_y[train_sel,:] 

test_x = test_x[test_sel,:,:]
test_y = test_y[test_sel,:] 

print('train_x shape:' , train_x.shape) 
print('train_y shape:'  , train_y.shape)     
print('test_x shape:' , test_x.shape)  
print('test_y shape:' , test_y.shape) 

n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
print('n_timesteps:' , n_timesteps) 
print('n_features:'  , n_features)     
print('n_outputs:' , n_outputs)    

def build_model(hp):
    #Esta estructura hay que retocarla
    model = Sequential()
    #Numero de nodos en la primera capa 
    hp_units = hp.Choice('input_unit',[32,64,128])
    model.add(LSTM(units = hp_units, input_shape=(n_timesteps, n_features)))
    model.add(RepeatVector(n_outputs))   
    #Numero de capas ocultas
    for i in range(hp.Int('n_layers', 1, 2)):
        units_i = hp.Choice(f'dense_{i}_units',[32,64,128])
        model.add(LSTM(units = hp_units, return_sequences=True))
    #Dropout
    hp_dropout = hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)
    model.add(Dropout(hp_dropout))
    model.add(TimeDistributed(Dense(units = hp_units)))
    
    model.add(TimeDistributed(Dense(n_outputs)))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(loss='mae', optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate))
    return model

tuner2= BayesianOptimization(
        build_model,
        objective='val_loss',
        max_trials=20,
        num_initial_points=2,
        seed = 123,
        overwrite=True
        )

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)

#Si esto da un error que dice algo de Oracle, se arregla borrando los json que hay en /untitle_project
tuner2.search(
        x=train_x,
        y=train_y,
        epochs=200,
        batch_size=128,
        validation_data=(test_x,test_y),
        verbose=2,
        shuffle=False,
        callbacks=[stop_early]
)

model2 = tuner2.get_best_models(num_models=1)[0]

model2.summary()

#model1.save('model_temp_mlp_01_varall_predsteps6_prev1_post6.h5')
#model1 = keras.models.load_model('model_temp_mlp_01_varall_predsteps6_prev1_post6.h5')

n_predict = 6 #Timesteps a predecir
prediction, test_y = predict(test_x, test_y, n_predict, n_obs_despues, model1)

pred_shape_0 = test.shape[0]
pred_shape_1 = n_predict 
# Ponemos la prediccion y el conjunto de test en 2D: muestras*timesteps
prediction = prediction.reshape(pred_shape_0, pred_shape_1)
test_y = test_y.reshape(pred_shape_0, pred_shape_1)

print('prediction shape:', prediction.shape)
print('test_y shape:', test_y.shape)

rmse = np.sqrt(mean_squared_error(test_y,prediction))
cvrmse = rmse/np.mean(test_y)*100
r2 = r2_score(test_y, prediction)
print('Test RMSE: %.3f' % rmse)
print('Test CVRMSE: %.3f' % cvrmse)
print('Test R squared: %.3f' % r2)

fig, axs = plt.subplots(figsize=(15,15), nrows = 8, ncols =8)
i=0
j=0
k=0
for i in range(0,8):
    for j, k in enumerate(range(k,k+8)):
        #print([i,j,k])
        axs[i,j].plot(test_y[k+i,:], label = 'real', marker='.')
        axs[i,j].plot(prediction[k+i,:], label = 'predict', marker='.')
        axs[i,j].set_ylim(20,35)
        axs[i,j].grid(linestyle = "--", alpha=0.5)
        axs[0,0].legend()
    fig.tight_layout()
plt.show()

fig, axs = plt.subplots(figsize=(15,15), nrows = 8, ncols =8)
i=0
j=0
k=0
for i in range(0,8):
    for j, k in enumerate(range(k,k+8)):
        #print([i,j,k])
        axs[i,j].plot(test_y[k+i+64,:], label = 'real', marker='.')
        axs[i,j].plot(prediction[k++i+64,:], label = 'predict', marker='.')
        axs[i,j].set_ylim(20,35)
        axs[i,j].grid(linestyle = "--", alpha=0.5)
        axs[0,0].legend()
    fig.tight_layout()
plt.show()

differences = np.abs((prediction - test_y).mean(axis=1)).round(0).astype(int)
differences

fig = plt.figure(figsize =(7, 5))
plt.grid(linestyle = "--", alpha=0.6)
plt.hist(differences, bins= [0,1,2,3,4,5,6,7,8,9], density=True, rwidth=0.8, align='mid')
#plt.xticks(np.arange(0, 10, 1))
plt.title("Histograma de diferencias entre real y predict")
plt.show()

In [ ]:
['dif_cons','cons_total','block','room', 'V12','V26','V5_0','V5_1','V5_2', 'Hora_1', 'Hora_2', 'Hora_3', 'Season_1', 'Season_2', 'Season_3', 'Season_4']